In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# LOAD DATA


NOTE: SS_cons symbols can be looked up in the infernal dokumentation

definition block:  bunch of adjacent < or > in the structure, now adding all other stems and a counter 

len + lennogap ,count   ,, length of alignments sometimes removing gaps, number of sequences in the alignment

aliX , ali1, ali2 , removing badly aligned sequences: 20%, 1 ,2   
calculation: rank sequences by outlier-points: if a column has 1 '.', or all but 1 '.' count an outlier-point

filtered: removing blocks of ~size~ len <3

flank: selected the positions that are within 5nuc of a block

cons, cov:  
calculation: mean([(max(Y,R)> .5 and (number of dots) else 0) / count])  
calculation: mean([(max(character) / count] and . becomes 0)  
calculation: cov/len 

stem length last-1: size of the second largest block 




In [2]:
debug = False

from sklearn.model_selection import train_test_split
import loadfiles
p,n  = loadfiles.loaddata("/home/pig/data",numneg=3000 if not debug else 200, pos='1' if debug else 'both', seed = 9)


['513-1063-0-0.sto', '463-3110-0-1.sto', '273-28605-2-1.sto', '480-9197-0-0.sto', '469-10338-0-0.sto']


# Make a pandas dataframe, 

In [3]:
import pandas as pd
import copy

def clean(di,oklist):
    for k in list(di.keys()):
        if k not in oklist:
            di.pop(k)
    return di

def makeXY(featurelist):
    asd = [ clean(e,featurelist) for e in copy.deepcopy(p+n) ]
    df = pd.DataFrame(asd)
    X= df.to_numpy()
    y= [1]*len(p)+[0]*len(n)
    return X,y,df


allfeatures = list(p[1].keys()) # the filenames are the last one and we dont need that (for now)
allfeatures.remove("name")
X,y,df = makeXY(allfeatures)

# check if is nan
from IPython.display import display, HTML
display(HTML(df[df.isna().any(axis=1)].head().to_html()))

,total conservation,total conservation +cov,total conservation_nuc,total conservation_nuc +cov,number of < blocks,number of ( blocks,number of [ blocks,number of { blocks,stem length smallest,stem length last-1,stem length max,stem cons,stem cons nuc,flank cons,flank cons nuc,perc stem,length,count,stem covariance,sloppy gu,sloppy all,allblockXalitotal conservation,allblockXalitotal conservation +cov,allblockXalitotal conservation_nuc,allblockXalitotal conservation_nuc +cov,allblockXali number of < blocks,allblockXali number of ( blocks,allblockXali number of [ blocks,allblockXali number of { blocks,allblockXali stem length smallest,allblockXali stem length last-1,allblockXali stem length max,allblockXali stem cons,allblockXali stem cons nuc,allblockXali flank cons,allblockXali flank cons nuc,allblockXaliperc stem,allblockXali length,allblockXali count,allblockXali stem covariance,allblockXali sloppy gu,allblockXali sloppy all,allblock1alitotal conservation,allblock1alitotal conservation +cov,allblock1alitotal conservation_nuc,allblock1alitotal conservation_nuc +cov,allblock1ali number of < blocks,allblock1ali number of ( blocks,allblock1ali number of [ blocks,allblock1ali number of { blocks,allblock1ali stem length smallest,allblock1ali stem length last-1,allblock1ali stem length max,allblock1ali stem cons,allblock1ali stem cons nuc,allblock1ali flank cons,allblock1ali flank cons nuc,allblock1aliperc stem,allblock1ali length,allblock1ali count,allblock1ali stem covariance,allblock1ali sloppy gu,allblock1ali sloppy all,allblock2alitotal conservation,allblock2alitotal conservation +cov,allblock2alitotal conservation_nuc,allblock2alitotal conservation_nuc +cov,allblock2ali number of < blocks,allblock2ali number of ( blocks,allblock2ali number of [ blocks,allblock2ali number of { blocks,allblock2ali stem length smallest,allblock2ali stem length last-1,allblock2ali stem length max,allblock2ali stem cons,allblock2ali stem cons nuc,allblock2ali flank cons,allblock2ali flank cons nuc,allblock2aliperc stem,allblock2ali length,allblock2ali count,allblock2ali stem covariance,allblock2ali sloppy gu,allblock2ali sloppy all,delblockalitotal conservation,delblockalitotal conservation +cov,delblockalitotal conservation_nuc,delblockalitotal conservation_nuc +cov,delblockali number of < blocks,delblockali number of ( blocks,delblockali number of [ blocks,delblockali number of { blocks,delblockali stem length smallest,delblockali stem length last-1,delblockali stem length max,delblockali stem cons,delblockali stem cons nuc,delblockali flank cons,delblockali flank cons nuc,delblockaliperc stem,delblockali length,delblockali count,delblockali stem covariance,delblockali sloppy gu,delblockali sloppy all,delblockXalitotal conservation,delblockXalitotal conservation +cov,delblockXalitotal conservation_nuc,delblockXalitotal conservation_nuc +cov,delblockXali number of < blocks,delblockXali number of ( blocks,delblockXali number of [ blocks,delblockXali number of { blocks,delblockXali stem length smallest,delblockXali stem length last-1,delblockXali stem length max,delblockXali stem cons,delblockXali stem cons nuc,delblockXali flank cons,delblockXali flank cons nuc,delblockXaliperc stem,delblockXali length,delblockXali count,delblockXali stem covariance,delblockXali sloppy gu,delblockXali sloppy all,delblock1alitotal conservation,delblock1alitotal conservation +cov,delblock1alitotal conservation_nuc,delblock1alitotal conservation_nuc +cov,delblock1ali number of < blocks,delblock1ali number of ( blocks,delblock1ali number of [ blocks,delblock1ali number of { blocks,delblock1ali stem length smallest,delblock1ali stem length last-1,delblock1ali stem length max,delblock1ali stem cons,delblock1ali stem cons nuc,delblock1ali flank cons,delblock1ali flank cons nuc,delblock1aliperc stem,delblock1ali length,delblock1ali count,delblock1ali stem covariance,delblock1ali sloppy gu,delblock1ali sloppy all,delblock2alitotal conservation,delblock2alitotal conservatio

# feature selection 

In [4]:
from sklearn.feature_selection import RFECV as rec
import pandas as pd
X = StandardScaler().fit_transform(X)
from sklearn.linear_model import Lasso
from skrebate import ReliefF as relief
randseed = 42
testsize=.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testsize, random_state=randseed) # USE THE SAME SEED AS BELOW! 


def myscore(y,yh):
    tpr = sum([ i==j for i,j in zip(y,yh) if i==1  ])/np.count_nonzero(y==1)
    tnr = sum([ i==j for i,j in zip(y,yh) if i==0  ])/np.count_nonzero(y==0)
    return ((2*tnr)+tpr)/3
    
def scorer(esti,X,y):
    yh = esti.predict(X)
    return myscore(y,yh)



def lasso(X,y,alpha=.06):
    mod = Lasso(alpha=alpha)
    mod.fit(X_train,y_train)
    return [b for a,b in zip(mod.coef_, df.columns) if a!=0]




reli=relief()
reli.fit(X,y)

def relief(X,y,param):
    #https://github.com/EpistasisLab/scikit-rebate
    return [ df.columns[top] for top in reli.top_features_[:param]]

featurelists = [ selector(X,y) for selector in [ lambda x,y: lasso(X,y,alpha=.05),  
                                                 lambda x,y: lasso(X,y,alpha=.01),
                                                 lambda x,y: relief(X,y,15),
                                                 lambda x,y: relief(X,y,30),
                                                 lambda x,y: relief(X,y,45)
                                               ]]

tmp =  pd.DataFrame([ [ 1 if f in featurelist else 0 for f in df.columns  ] for featurelist in featurelists],columns = df.columns)
display(HTML(tmp.loc[:, (tmp != 0).any(axis=0)].to_html()))

/home/pig/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


,number of < blocks,number of ( blocks,number of [ blocks,flank cons,length,count,stem covariance,sloppy all,allblockXali number of < blocks,allblockXali stem length last-1,allblockXali stem length max,allblockXali flank cons,allblockXali length,allblockXali count,allblockXali sloppy all,allblock1ali number of < blocks,allblock1ali number of ( blocks,allblock1ali stem length smallest,allblock1ali stem length max,allblock1ali stem cons nuc,allblock1ali length,allblock1ali count,allblock2alitotal conservation_nuc +cov,allblock2ali number of < blocks,allblock2ali number of ( blocks,allblock2ali stem length last-1,allblock2ali stem cons nuc,allblock2ali length,allblock2ali count,allblock2ali sloppy gu,delblockalitotal conservation_nuc +cov,delblockali number of < blocks,delblockali number of ( blocks,delblockali number of { blocks,delblockali flank cons,delblockali length,delblockali count,delblockali sloppy gu,delblockali sloppy all,delblockXali number of < blocks,delblockXali number of ( blocks,delblockXali stem length smallest,delblockXali stem length max,delblockXali stem cons nuc,delblockXali flank cons,delblockXali length,delblockXali count,delblockXali stem covariance,delblockXali sloppy gu,delblock1ali number of < blocks,delblock1ali number of ( blocks,delblock1ali number of [ blocks,delblock1ali stem length smallest,delblock1ali stem length max,delblock1ali stem cons nuc,delblock1ali length,delblock1ali count,delblock2ali number of < blocks,delblock2ali number of ( blocks,delblock2ali stem length max,delblock2ali stem cons,delblock2ali length,delblock2ali count,delblock2ali stem covariance,delblock2ali sloppy gu
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,1,1,1,1,0,0,1,1,0,1,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,1,0,1,0,0,0,0,1,1,1,0,1,1
2,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0
3,1,1,0,1,1,1,0,0,1,0,1,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,1,0,1,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,1,0,0,1,1,1,0
4,1,1,0,1,1,1,0,0,1,0,1,1,1,1,0,1,1,0,1,0,1,1,0,1,1,0,1,1,1,1,0,1,1,0,0,1,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1


# check performance for various classifiers

In [5]:

import draw
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
names = ["Nearest Neighbors","Linear SVM", "RBF SVM",
         #"Gaussian Process", # 2 slow
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA",'extra trees','gradient boosting']
res = []

for FEATURELIST in featurelists:  # loop over all the selectors 
    
    # make some data 
    X,y,df = makeXY(FEATURELIST)
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testsize, random_state=randseed) # USE THE SAME SEED AS BELOW! 

    classifiers = [
        KNeighborsClassifier(5),
        SVC(kernel="linear",class_weight='balanced', C=0.025),
        SVC(gamma=2, C=1,class_weight='balanced'),
        #SVC(),
        #GaussianProcessClassifier(1.0 * RBF(1.0)),
        DecisionTreeClassifier(max_depth=5,class_weight='balanced',min_samples_leaf=4),
        RandomForestClassifier(max_depth=9, n_estimators=30, class_weight='balanced'),
        #RandomForestClassifier(max_depth=5, n_estimators=25, max_features=5,class_weight='balanced'),
        MLPClassifier(alpha=.001, max_iter=2000),
        AdaBoostClassifier(DecisionTreeClassifier(max_depth=3)),
        GaussianNB(),
        QuadraticDiscriminantAnalysis(),
        ExtraTreesClassifier(),
        GradientBoostingClassifier()
        ]

    def score(clf):
        clf.fit(X_train, y_train)
        return scorer(clf,X_test,np.array(y_test))
        
    res.append( [score(clf) for clf in classifiers] )
    
display(HTML(pd.DataFrame(res,columns=names).to_html()))

/home/pig/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/pig/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/pig/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/pig/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/pig/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/pig/.local/lib/python3.6/site-packages/

,Nearest Neighbors,Linear SVM,RBF SVM,Decision Tree,Random Forest,Neural Net,AdaBoost,Naive Bayes,QDA,extra trees,gradient boosting
0,0.680678,0.677527,0.665817,0.655126,0.687662,0.698101,0.697753,0.697698,0.682550,0.672871,0.700599
1,0.755934,0.775516,0.667832,0.669185,0.789237,0.809085,0.772937,0.718972,0.695307,0.745942,0.785569
2,0.682435,0.674451,0.647195,0.618387,0.675077,0.703560,0.674795,0.678278,0.657685,0.692830,0.698045
3,0.706966,0.714100,0.666667,0.663447,0.743803,0.745372,0.717567,0.694891,0.508692,0.729569,0.740522
4,0.716641,0.717189,0.666667,0.684242,0.753329,0.749571,0.721582,0.710004,0.519786,0.726943,0.746109
